In [1]:
%%writefile example2.txt
first
second line
the third line
then a forth line 

Overwriting example2.txt


In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext()

In [4]:
sc.textFile('example2.txt')

example2.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [5]:
text_rdd = sc.textFile('example2.txt')

In [6]:
words = text_rdd.map(lambda line: line.split())

In [7]:
words.collect()

[['first'],
 ['second', 'line'],
 ['the', 'third', 'line'],
 ['then', 'a', 'forth', 'line']]

In [8]:
text_rdd.collect()

['first', 'second line', 'the third line', 'then a forth line ']

In [9]:
text_rdd.flatMap(lambda line: line.split()).collect()

['first',
 'second',
 'line',
 'the',
 'third',
 'line',
 'then',
 'a',
 'forth',
 'line']

In [10]:
%%writefile services.txt
#EventId    Timestamp    Customer   State    ServiceID    Amount
201       10/13/2017      100       NY       131          100.00
204       10/18/2017      700       TX       129          450.00
202       10/15/2017      203       CA       121          200.00
206       10/19/2017      202       CA       131          500.00
203       10/17/2017      101       NY       173          750.00
205       10/19/2017      202       TX       121          200.00

Overwriting services.txt


In [11]:
services = sc.textFile('services.txt')

In [12]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [15]:
services.map(lambda line: line.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [16]:
services.map(lambda line: line[1:] if line[0] =='#' else line).collect()

['EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [17]:
clean = services.map(lambda line: line[1:] if line[0] =='#' else line)

In [19]:
clean = clean.map(lambda line: line.split())

In [20]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [25]:
clean.map(lambda line: (line[3], line[-1])).collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [26]:
pairs = clean.map(lambda line: (line[3], line[-1]))

In [29]:
rekey = pairs.reduceByKey(lambda amt1, amt2: float(amt1) + float(amt2))

In [30]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [31]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [42]:
#Grab State and Amount: (State, Amount)
step1 = clean.map(lambda st: (st[3], st[-1]))
#Reduce by Key
step2 = step1.reduceByKey(lambda amt1, amt2: float(amt1) + float(amt2))
#Get rid of State, Amount titles
step3 = step2.filter(lambda x: not x[0] == 'State')
#Sort the result by Amount
step4 = step3.sortBy(lambda stAmount: stAmount[1], ascending=False)
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]